# Chapter 8: Text Generation with Recurrent Neural Networks (RNNs)


This chapter covers

* The idea behind RNNs and why they can handle sequential data 
* Character tokenization, word tokenization, and subword tokenization
* How word embedding works
* Building and training an RNN to generate text 
* Using temperature and top-K sampling to control the creativeness of text generation

So far in this book, you have learned to generate shapes, numbers, and images. Starting from this chapter, we’ll focus mainly on text generation. Generating text is often considered the holy grail of generative AI for several compelling reasons. Human language is incredibly complex and nuanced. It involves not only understanding grammar and vocabulary but also context, tone, and cultural references. Successfully generating coherent and contextually appropriate text is a significant challenge that requires deep understanding and processing of language. 

As humans, we primarily communicate through language. AI that can generate human-like text can interact more naturally with users, making technology more accessible and user-friendly. Text generation has many applications, from automating customer service responses to creating entire articles, scripting for games and movies, aiding in creative writing, and even building personal assistants. The potential impact across industries is enormous.

In this chapter, you’ll make your first attempt at building and training models to generate text. You’ll learn to tackle three main challenges in modeling text generation. Firstly, text is sequential data, meaning the arrangement of words in a sentence is crucial. Predicting outcomes for sequences is challenging due to their sensitive ordering. Altering the sequence of elements changes their meaning. Secondly, the meaning of a certain part of the text depends on elements that appeared much earlier in the text. Understanding and modeling these long-range dependencies is essential for generating coherent text. Lastly, Human language is ambiguous and context-dependent. Training a model to understand nuances, sarcasm, idioms, and cultural references to generate contextually accurate text is challenging.

You'll explore a specific neural network adept at handling sequential data, such as text or time series: the recurrent neural network (RNN). Unlike typical neural networks with one-directional neuron connections—from the input layer through hidden layers to the output layer—RNNs feature bi-directional connections, allowing feedback from later to earlier layers. We’ll focus on one variant of RNN, Long Short-Term Memory (LSTM) networks, which can recognize both short-term and long-term data patterns in sequential data like text. LSTM models use a hidden state to capture information in previous time steps. Therefore, a trained LSTM model can produce coherent text based on the context. 

The style of the generated text depends on the training data. You’ll use the text from the novel Anna Karenina to train an LSTM model. Since neural networks like an LSTM cannot accept text as input directly, you’ll learn to break down text into tokens (individual words or parts of words), a process known as tokenization. You’ll then create a dictionary to map each unique token into an integer (i.e., an index). Based on this dictionary, you’ll convert the text into a long sequence of integers, ready to be fed into a neural network. 

You’ll use sequences of indexes of a certain length as the input to train the LSTM model. You shift the sequence of inputs by one token to the right and use it as the output: you are effectively training the model to predict the next word in a sentence. A simplified example is as follows: the indexes corresponding to the four words “a frog has four” are used as input and those corresponding to “frog has four legs” as output. The model learns to use the word “a” to predict “frog”, to use “a frog” to predict “has”, and so on. Finally, the model uses “a frog has four” to predict the word “legs.” This is the so-called sequence-to-sequence prediction problem in natural language processing (NLP) and you’ll see it again in later chapters. 

Once the LSTM is trained, you’ll use it to generate text. You feed a prompt (part of a sentence such as “Anna and the”) to the trained model. The model predicts what’s the most likely next token and appends the selected token to your prompt. The new prompt is used again as the input; we feed this new prompt to the model again to predict the next token. The process continues until the prompt reaches a certain length. You’ll witness the trained LSTM model generating grammatically correct and coherent text, with a style matching that of the original novel. 

Finally, you also learn how to control the creativeness of the generated text by using temperature and top-K sampling. Temperature controls the randomness of the predictions of the trained model by scaling the logits (the inputs to the softmax function for probability calculation). A high temperature makes the generated text more creative while a low temperature makes the text more confident and predictable. Top-K sampling is a method where you select the K words with the highest probabilities instead of all words from the vocabulary. A small value of K leads to the selection of highly likely words in each step and this, in turn, makes the generated text less creative and more coherent.

# 1.    Introduction to recurrent neural networks (RNNs)
# 2.	Fundamentals of Natural Language Processing (NLP)

In [1]:
# character tokenization
text="It is unbelievably good!"
tokens=list(text)
print(tokens)

['I', 't', ' ', 'i', 's', ' ', 'u', 'n', 'b', 'e', 'l', 'i', 'e', 'v', 'a', 'b', 'l', 'y', ' ', 'g', 'o', 'o', 'd', '!']


In [2]:
# exercise 8.1
text="Hi, there!"
tokens=list(text)
print(tokens)

['H', 'i', ',', ' ', 't', 'h', 'e', 'r', 'e', '!']


In [3]:
# word tokenization
text="It is unbelievably good!"
text=text.replace("!"," !")
tokens=text.split(" ")
print(tokens)

['It', 'is', 'unbelievably', 'good', '!']


In [4]:
# exercise 8.2
text="Hi, there!"
for x in list(",!"):
    text=text.replace(f"{x}",f" {x}")
tokens=text.split(" ")
print(tokens)

['Hi', ',', 'there', '!']


# 3.	Prepare data to train the LSTM model
We'll use the text file of Anna Karenina in one of Carlos Lara's GitHub repositories. Go to the link https://github.com/LeanManager/NLP-PyTorch/tree/master/data to download the text file and save it as *anna.txt* in the folder /files/ on your computer. After that, open the file and delete everything after line 39888, which says "END OF THIS PROJECT GUTENBERG EBOOK ANNA KARENINA." 

## 3.1	Download the clean up the text


In [5]:
with open("files/anna.txt","r") as f:
    text=f.read()
words=text.split(" ") 
print(words[:20])

['Chapter', '1\n\n\nHappy', 'families', 'are', 'all', 'alike;', 'every', 'unhappy', 'family', 'is', 'unhappy', 'in', 'its', 'own\nway.\n\nEverything', 'was', 'in', 'confusion', 'in', 'the', "Oblonskys'"]


In [6]:
print(set(text.lower()))

{'f', 'd', '"', 'h', 'b', '1', '3', '6', '8', 'o', 'i', '_', 'r', ',', ')', '9', 'a', 'j', '`', '5', '0', 'y', 'q', 'x', '2', 'g', 'z', 'p', "'", '?', 'm', 'v', ' ', '\n', 'e', '-', ':', 'w', 'n', '.', ';', '4', 'k', '(', 'u', '!', '7', 'c', 's', 't', 'l'}


In [7]:
clean_text=text.lower().replace("\n", " ")
clean_text=clean_text.replace("-", " ")
for x in ",.:;?!$()/_&%*@'`":
    clean_text=clean_text.replace(f"{x}", f" {x} ")
clean_text=clean_text.replace('"', ' " ') 
text=clean_text.split()

In [8]:
from collections import Counter   
word_counts = Counter(text)    

# get unique words
words=sorted(word_counts, key=word_counts.get,
                      reverse=True) 
print(words[:10])

[',', '.', 'the', '"', 'and', 'to', 'of', 'he', "'", 'a']


In [9]:
text_length=len(text)
num_unique_words=len(words)
print(f"the text contains {text_length} words")
print(f"there are {num_unique_words} unique tokens")  
word_to_int={v:k for k,v in enumerate(words)} 
int_to_word={k:v for k,v in enumerate(words)}
print({k:v for k,v in word_to_int.items() if k in words[:10]})
print({k:v for k,v in int_to_word.items() if v in words[:10]})

the text contains 437098 words
there are 12778 unique tokens
{',': 0, '.': 1, 'the': 2, '"': 3, 'and': 4, 'to': 5, 'of': 6, 'he': 7, "'": 8, 'a': 9}
{0: ',', 1: '.', 2: 'the', 3: '"', 4: 'and', 5: 'to', 6: 'of', 7: 'he', 8: "'", 9: 'a'}


In [10]:
print(text[0:20])
wordidx=[word_to_int[w] for w in text]  
print([word_to_int[w] for w in text[0:20]])  

['chapter', '1', 'happy', 'families', 'are', 'all', 'alike', ';', 'every', 'unhappy', 'family', 'is', 'unhappy', 'in', 'its', 'own', 'way', '.', 'everything', 'was']
[208, 2755, 280, 2981, 83, 31, 2419, 35, 202, 685, 362, 38, 685, 10, 236, 147, 166, 1, 149, 12]


## 3.2	Create batches of training data


In [11]:
import torch

seq_len=100  
xys=[]
for n in range(0, len(wordidx)-seq_len-1):
    x = wordidx[n:n+seq_len]
    y = wordidx[n+1:n+seq_len+1]
    xys.append((torch.tensor(x),(torch.tensor(y))))

In [12]:
from torch.utils.data import DataLoader

torch.manual_seed(42)
batch_size=32
loader = DataLoader(xys, batch_size=batch_size, shuffle=True)

x,y=next(iter(loader))
print(x)
print(y)
print(x.shape,y.shape)

tensor([[  39,   31,    2,  ...,  688,  142,    7],
        [ 156, 5293,    0,  ...,   38,  330,    0],
        [   3,   97,    0,  ...,    0, 1774,   34],
        ...,
        [  16,  156,    9,  ...,  113,    5,  533],
        [   3,    4,   31,  ...,   98,    5,   98],
        [ 289,   19,   23,  ...,    9,  828,  550]])
tensor([[  31,    2, 2727,  ...,  142,    7,    0],
        [5293,    0,   16,  ...,  330,    0,    3],
        [  97,    0,    4,  ..., 1774,   34,    3],
        ...,
        [ 156,    9,  489,  ...,    5,  533,   27],
        [   4,   31,   25,  ...,    5,   98,    1],
        [  19,   23,    1,  ...,  828,  550,    1]])
torch.Size([32, 100]) torch.Size([32, 100])


# 4. Build and Train the LSTM Model


## 4.1	Build an LSTM model


In [13]:
import torch
from torch import nn
device="cuda" if torch.cuda.is_available() else "cpu"
class WordLSTM(nn.Module):
    def __init__(self, input_size=128, n_embed=128,
             n_layers=3, drop_prob=0.2):
        super().__init__()
        self.input_size=input_size
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_embed = n_embed
        vocab_size=len(word_to_int)
        self.embedding=nn.Embedding(vocab_size,n_embed)
        self.lstm = nn.LSTM(input_size=self.input_size,
            hidden_size=self.n_embed,
            num_layers=self.n_layers,
            dropout=self.drop_prob,batch_first=True)
        self.fc = nn.Linear(input_size, vocab_size)
    def forward(self, x, hc):
        embed=self.embedding(x)
        x, hc = self.lstm(embed, hc)
        x = self.fc(x)
        return x, hc        
    def init_hidden(self, n_seqs):
        weight = next(self.parameters()).data
        return (weight.new(self.n_layers,
                           n_seqs, self.n_embed).zero_(),
                weight.new(self.n_layers,
                           n_seqs, self.n_embed).zero_()) 

In [14]:
model=WordLSTM().to(device)
print(model)

WordLSTM(
  (embedding): Embedding(12778, 128)
  (lstm): LSTM(128, 128, num_layers=3, batch_first=True, dropout=0.2)
  (fc): Linear(in_features=128, out_features=12778, bias=True)
)


In [15]:
lr=0.0001
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
loss_func = nn.CrossEntropyLoss()

## 4.2	Train the LSTM model


In [16]:
model.train()

for epoch in range(50):
    tloss=0
    sh,sc = model.init_hidden(batch_size)
    for i, (x,y) in enumerate(loader):    
        if x.shape[0]==batch_size:
            inputs, targets = x.to(device), y.to(device)
            optimizer.zero_grad()
            output, (sh,sc) = model(inputs, (sh,sc))
            loss = loss_func(output.transpose(1,2),targets)
            sh,sc=sh.detach(),sc.detach()
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), 5)
            optimizer.step()
            tloss+=loss.item()
        if (i+1)%1000==0:
            print(f"at epoch {epoch} iteration {i+1}\
            average loss = {tloss/(i+1)}")

In [17]:
import pickle

torch.save(model.state_dict(),"files/wordLSTM.pth")
with open("files/word_to_int.p","wb") as fb:    
    pickle.dump(word_to_int, fb) 

# 5	Generate text with the trained LSTM model


## 5.1	Generate text by predicting the next token

In [18]:
import pickle
model.load_state_dict(torch.load("files/wordLSTM.pth"))
with open("files/word_to_int.p","rb") as fb:    
    word_to_int = pickle.load(fb)      
int_to_word={v:k for k,v in word_to_int.items()}

In [19]:
import numpy as np
def sample(model, prompt, length=200):
    model.eval()
    text = prompt.lower().split(' ')
    hc = model.init_hidden(1)
    length = length - len(text)
    for i in range(0, length):
        # if the text length is less than 50, use text to predict 
        if len(text)<=50:
            x = torch.tensor([[word_to_int[w] for w in text]])
        # otherwise use the last 50 tokens to predict
        else:
            x = torch.tensor([[word_to_int[w] for w in text[-50:]]])            
        inputs = x.to(device)
        output, hc = model(inputs, hc)
        logits = output[0][-1]
        p = nn.functional.softmax(logits, dim=0).detach().cpu().numpy()
        idx = np.random.choice(len(logits), p=p)
        text.append(int_to_word[idx])
    text=" ".join(text)
    for m in ",.:;?!$()/_&%*@'`":
        text=text.replace(f" {m}", f"{m} ")
    text=text.replace('"  ', '"')   
    text=text.replace("'  ", "'")  
    text=text.replace('" ', '"')   
    text=text.replace("' ", "'")     
    return text  

In [20]:
import torch
torch.manual_seed(42)
np.random.seed(42)
print(sample(model, prompt='Anna and the prince'))  

anna and the prince did not forget what he had not spoken.  when the softening barrier was not so long as he had talked to his brother,  all the hopelessness of the impression.  "official tail,  a man who had tried him,  though he had been able to get across his charge and locked close,  and the light done that spring was still more.  behind the bone of the very bird could not act,  but he grew thoughtful.  the boy that they were reconciled to go into the station.  he knew that the words of his brother had inner brother that levin was a great place;  but with extreme regret and until he liked quite glad,  that he said something,  though a new brother she remembered what she had said)  read that she was feeling.  but she was turning back at the angry seat of fare,  he kept up so as as he spoke.  "to take all the books,  then he won't go on,  "said levin,  rubbing his legs in the curve of his hands by


## 5.2	Temperature and top-K sampling in text generation

In [21]:
def generate(model, prompt, top_k=None, 
             length=200, temperature=1):
    model.eval()
    text = prompt.lower().split(' ')
    hc = model.init_hidden(1)
    length = length - len(text)    
    for i in range(0, length):
        # if the text length is less than 50, use text to predict 
        if len(text)<=50:
            x = torch.tensor([[word_to_int[w] for w in text]])
        # otherwise use the last 50 tokens to predict
        else:
            x = torch.tensor([[word_to_int[w] for w in text[-50:]]])    
        inputs = x.to(device)
        output, hc = model(inputs, hc)
        logits = output[0][-1]
        # scale the logits with the temperature 
        logits = logits/temperature
        p = nn.functional.softmax(logits, dim=0).detach().cpu()    
        if top_k is None:
            idx = np.random.choice(len(logits), p=p.numpy())
        # top-K sampling
        else:
            ps, tops = p.topk(top_k)
            ps=ps/ps.sum()
            idx = np.random.choice(tops, p=ps.numpy())          
        text.append(int_to_word[idx])
    text=" ".join(text)
    for m in ",.:;?!$()/_&%*@'`":
        text=text.replace(f" {m}", f"{m} ")
    text=text.replace('"  ', '"')   
    text=text.replace("'  ", "'")  
    text=text.replace('" ', '"')   
    text=text.replace("' ", "'")     
    return text   

In [22]:
# next token using default setting
prompt="I ' m not going to see"
torch.manual_seed(42)
np.random.seed(42)
for _ in range(10):
    print(generate(model, prompt, top_k=None, 
         length=len(prompt.split(" "))+1, temperature=1)) 

i'm not going to see you
i'm not going to see those
i'm not going to see me
i'm not going to see you
i'm not going to see her
i'm not going to see her
i'm not going to see the
i'm not going to see my
i'm not going to see you
i'm not going to see me


In [23]:
# next token using conservative predictions
prompt="I ' m not going to see"
torch.manual_seed(42)
np.random.seed(42)
for _ in range(10):
    print(generate(model, prompt, top_k=3, 
         length=len(prompt.split(" "))+1, temperature=0.5)) 

i'm not going to see you
i'm not going to see the
i'm not going to see her
i'm not going to see you
i'm not going to see you
i'm not going to see you
i'm not going to see you
i'm not going to see her
i'm not going to see you
i'm not going to see her


In [24]:
torch.manual_seed(42)
np.random.seed(42)
print(generate(model, prompt='Anna and the prince',
               top_k=3,
               temperature=0.5)) 

anna and the prince had no milk.  but,  "answered levin,  and he stopped.  "i've been skating to look at you all the harrows,  and i'm glad. . .  ""no,  i'm going to the country.  ""no,  no,  i'll go to the counting house.  ""well,  sir,  i'll go.  ""yes,  sir.  ""yes,  sir.  ""oh,  yes,  sir.  ""oh,  yes,  parmesan.  ""well,  well,  what do you think about it?  ""oh,  yes,  parmesan.  ""oh,  well,  what do you think?  ""no,  "said stepan arkadyevitch,  getting up.  "i'll send to the bee house,  or to keep up the dogs,  where the diplomatic process. . . .  ""no,  putty!  "said levin,  and it was not merely the same thing to him.  "i'm


In [25]:
# next token using creative predictions
prompt="I ' m not going to see"
torch.manual_seed(42)
np.random.seed(42)
for _ in range(10):
    print(generate(model, prompt, top_k=None, 
         length=len(prompt.split(" "))+1, temperature=2)) 

i'm not going to see them
i'm not going to see scarlatina
i'm not going to see behind
i'm not going to see us
i'm not going to see it
i'm not going to see it
i'm not going to see a
i'm not going to see misery
i'm not going to see another
i'm not going to see seryozha


In [26]:
torch.manual_seed(42)
np.random.seed(42)
print(generate(model, prompt='Anna and the prince',
               top_k=None,
               temperature=2)) 

anna and the prince took sheaves covered suddenly people.  "pyotr marya borissovna,  propped mihail though her son will seen how much evening her husband;  if tomorrow she liked great time too.  "adopted heavens details for it women from this terrible,  admitting this touching all feeling utterly different evoked having studiously ceasing eager rejoicing of what he did only first harm at what answered her moist,  protestant cold? .  vassenka nor entered marya words with his pockets;  they were making their comrades almost gold just that four space has alexander fresh higher ladies,  a or quarrel from talk wide.  anna avoided this.  on were naughty frou oblonsky announced several talk died here tomorrow within their two years shooting bright scattered,  delightful town the whole girl gold and clear difficulties or occupations husbands so talking but idea,  especially long,  though served for you feeling extraneous idea how irritated all obviously never clearly simply wanted to respected